In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.sparse as sp


In [2]:
# Description: Crank-Nicholson Scheme for 2D Heat Equation





# Crank-Nicholson Scheme


# Parameters
width = 1.0     # lower-end of the rod
height = 1.0    # upper-end of the rod
N = 100      # number of width divisions
K = 100    # number of height divisions
dx = width / N  # spatial step size
dy = height / K
C = 1.0 # thermal diffusivity
dt = 0.01   # time step size
# crank-nicolson constant
i_c = 0.5 
theta = i_c * C*dt/(dx**2)
alpha = (1 - i_c) * C * dt /(dx**2)
T = 0.1     # total time
M = int(T / dt)  # number of time steps

# Spatial grid
x = np.linspace(0, width, N)
y = np.linspace(0, height, K)
X,Y = np.meshgrid(x, y)
u = np.sin(np.pi * X)* np.sin(np.pi * Y)

# diagonal length is defined by the number of grid points
i_grid_size = (K-1) * (N-1)
# create the coefficient matrix
# diagonal of a 1-4*Lambda 

offdiagonals = np.zeros(i_grid_size - 1)
offdiagonals[(np.arange(1, i_grid_size)) % (K -1) != 0] = -theta
# np.fill(diagonals[(np.arange(1, i_grid_size)) % (K -1) != 0])

# diagonals.fill(-theta)
diagsVec = [-theta , offdiagonals, 1 + 4 * theta, offdiagonals, -theta]
# diags[:, [0, 1, 3, 4]] = -theta
# diags[:,2] = 1 + 4 * theta
# #zeros from BCs
offsets = [-N+1, -1, 0, 1, N-1]


A = scipy.sparse.diags(diagsVec, offsets, shape=((K-1) * (N-1), (K-1) * (N-1))).toarray()
print(A)





print(u)



# Time iteration
#PUT 3D ARRAY
# u_hist = [u.copy()] # history of u for plotting
u_hist = np.zeros(shape=(M + 1, N, K))
u_hist[0,:,:] = u
for k in range(1, M+1):
    b = np.zeros((K-1) * (N-1))
    # print("u_n: ", u)
    for i in range(1, N-1):
        for i in range(1, K-1): 
            index = (i-1) * (N-1)+ i -1  # Adjusted index calculation for interior points
            # b[index] = alpha * u[i+1, j] + alpha * u[i, j + 1] + (1- 4*alpha) * u[i, j] \
            # + alpha * u[i - 1, j] + alpha * u[i, j -1]
            b[index] = (1 - 4 * alpha) * u[i,i] + alpha * (u[i+1,i] + u[i,i+1] + u[i-1,i] + u[i,i-1])
   
    u_new = sp.linalg.spsolve(A, b)
    u_new_2d = np.zeros_like(u)
    index = 0
    # for i in range(1, N):
    #     for j in range(1, K):
    #         u_new_2d[i, j] = u_new[index]
    #         index += 1


    u_new_2d[1:N, 1:K] = u_new.reshape(N-1, K-1)

    u = u_new_2d
    u_hist[k,:,:] = u_new_2d

u_hist = np.array(u_hist)




[[201. -50.   0. ...   0.   0.   0.]
 [-50. 201. -50. ...   0.   0.   0.]
 [  0. -50. 201. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ... 201. -50.   0.]
 [  0.   0.   0. ... -50. 201. -50.]
 [  0.   0.   0. ...   0. -50. 201.]]
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00666176e-03 2.01230991e-03 ... 2.01230991e-03
  1.00666176e-03 3.88555122e-18]
 [0.00000000e+00 2.01230991e-03 4.02259358e-03 ... 4.02259358e-03
  2.01230991e-03 7.76719002e-18]
 ...
 [0.00000000e+00 2.01230991e-03 4.02259358e-03 ... 4.02259358e-03
  2.01230991e-03 7.76719002e-18]
 [0.00000000e+00 1.00666176e-03 2.01230991e-03 ... 2.01230991e-03
  1.00666176e-03 3.88555122e-18]
 [0.00000000e+00 3.88555122e-18 7.76719002e-18 ... 7.76719002e-18
  3.88555122e-18 1.49975978e-32]]


/var/folders/rg/cmtcr_3n3g5bk2x49f7jdpdw0000gn/T/ipykernel_94164/1873199121.py:75: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  u_new = sp.linalg.spsolve(A, b)


In [3]:
# def findAi(S2, S1, S0, dt, r, sigma, dS_plus, dS_minus, grid_size):
#     Sph = S2 - S1
#     Smh = S1 - S0

#     ai = (sigma**2 * S1**2 / (Sph * dS_plus[:-1]) + r * S1 / dS_plus[:-1]) * dt * 0.5
#     ci = (sigma**2 * S1**2 / (Smh * dS_minus[:-1]) - r * S1 / dS_minus[:-1]) * dt * 0.5
#     ai = np.append(ai, (sigma**2 * S1**2 / (Sph[-1] * dS_plus[-1]) + r * S1 / dS_plus[-1]) * dt * 0.5)
#     ci = np.append(ci, (sigma**2 * S1**2 / (Smh[-1] * dS_minus[-1]) - r * S1 / dS_minus[-1]) * dt * 0.5)
#     bi = -(ai + ci + r * dt)
    
#     return scipy.sparse.diags([ci, bi, ai], [-1, 0, 1], shape=(grid_size, grid_size))


# def approximate_black_scholes(s_grid,  K, T, r, sigma, t_i = 1000, option_type = 'call'):
# 	dt = T / t_i
# 	N = s_grid.shape[0]
# 	v = np.zeros((t_i, N))
# 	v[0, :] = np.maximum(s_grid - K, 0) if option_type == 'call' else np.maximum(K - s_grid, 0)
# 	dS_plus = np.diff(s_grid)  # dS[i] = s_grid[i+1] - s_grid[i]
# 	dS_minus = np.insert(dS_plus[:-1], 0, dS_plus[0]) 

# 	for n in range(t_i - 1):
# 		if option_type == 'call':
# 			v[n + 1, 0] = 0  # Call option value at nearly zero asset price
# 			v[n + 1, -1] = s_grid[-1] - K * np.exp(-r * (T - dt * (n + 1)))
# 		elif option_type == 'put':
# 			v[n + 1, 0] = K * np.exp(-r * (T - dt * (n + 1)))
# 			v[n + 1, -1] = 0  # Put option value at the upper boundary
# 		Ai = findAi(s_grid[2:], s_grid[1:-1], s_grid[: -2], dt, r, sigma, dS_plus, dS_minus, N - 2)
# 		Ai_dense = Ai.toarray()
#         # Use numpy.linalg.solve to solve the linear system
# 		v[n + 1, 1:-1] = np.linalg.solve(Ai_dense, v[n, 1:-1])
		
	
# 	return v

In [30]:
import math

S_0 = 100
K = 100
T = 1
r = 0.05
sigma = 0.2
d1 = (math.log(S_0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
d2 = d1 - sigma * math.sqrt(T)

call_option_value = (S_0 * math.exp(-r * T) * (math.erfc(d1 * math.sqrt(2)) / 2) - K * math.exp(-r * T) * (math.erfc(d2 * math.sqrt(2)) / 2))

print("Analytical solution for European call option:", call_option_value)


Analytical solution for European call option: -13.329095232525415


In [38]:
def findAi(S2, S1, S0, dt, r, sigma, dS_plus, dS_minus, grid_size):
    Sph = S2 - S1
    Smh = S1 - S0

    ai = (sigma**2 * S1**2 / (Sph * dS_plus[:-1]) + r * S1 / dS_plus[:-1]) * dt * 0.5
    ci = (sigma**2 * S1**2 / (Smh * dS_minus[:-1]) - r * S1 / dS_minus[:-1]) * dt * 0.5
    ai = np.append(ai, (sigma**2 * S1**2 / (Sph[-1] * dS_plus[-1]) + r * S1 / dS_plus[-1]) * dt * 0.5)
    ci = np.append(ci, (sigma**2 * S1**2 / (Smh[-1] * dS_minus[-1]) - r * S1 / dS_minus[-1]) * dt * 0.5)
    bi = -(ai + ci + r * dt)
    return scipy.sparse.diags([ci, bi, ai], [-1, 0, 1], shape=(grid_size, grid_size))


def approximate_black_scholes(s_grid, K, T, rho, sigma, t_i = 1000, option_type = 'call'):
    dt = T / t_i
    N = s_grid.shape[0]
    v = np.zeros((t_i, N))
    v[0, :] = np.maximum(s_grid - K, 0) if option_type == 'call' else np.maximum(K - s_grid, 0)
    dS_plus = np.diff(s_grid)  # dS[i] = s_grid[i+1] - s_grid[i]
    dS_minus = np.insert(dS_plus[:-1], 0, dS_plus[0])

    for n in range(t_i - 1):
        
        # Convert the sparse matrix to a dense array
        
        # Use numpy.linalg.solve to solve the linear system
        Ai = findAi(s_grid[2:], s_grid[1:-1], s_grid[: -2], dt, rho, sigma, dS_plus, dS_minus, N - 2)
        v[n + 1, 1:-1] = sp.linalg.spsolve(Ai, v[n, 1:-1])
        if option_type == 'call':
            v[n + 1, 0] = 0  # Call option value at nearly zero asset price
            v[n + 1, -1] = np.max(K - v[n, -1], 0) * np.exp(-rho * (T - dt * (n + 1)))
        elif option_type == 'put':
            v[n + 1, 0] = K * np.exp(-rho * (T - dt * (n + 1)))
            v[n + 1, -1] = 0  # Put option value at the upper boundary
        
        
    return v

In [39]:
S_0 = 100
K = 100
T = 1
r = 0.05
sigma = 0.2
t_i = 10000
s_grid = np.linspace(0, 200, 200)

v = approximate_black_scholes(s_grid, K, T, r, sigma, t_i)
print("Numerical solution for European call option:", v[-1, -1])

/var/folders/rg/cmtcr_3n3g5bk2x49f7jdpdw0000gn/T/ipykernel_94164/2369033926.py:27: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  v[n + 1, 1:-1] = sp.linalg.spsolve(Ai, v[n, 1:-1])


Numerical solution for European call option: 49.999937499921934
